In [1]:
from get_match_data import wc_rankings, wc_start, get_data
from precompute import get_groups
from prediction_model import get_model
from simulation import Simulation
from evaluation import common_winners

In [2]:
YEAR = 2018  # year for the model
rankings, data = get_data(YEAR)
logreg, match_data = get_model(data, report=True)

Mean Validation accuracy: 0.7466066164815621
Test data model accuracy: 0.754321554229769

               precision    recall  f1-score   support

       False       0.78      0.84      0.81      3980
        True       0.71      0.62      0.67      2557

    accuracy                           0.75      6537
   macro avg       0.74      0.73      0.74      6537
weighted avg       0.75      0.75      0.75      6537

Baseline {predict higher ranked team}:
               precision    recall  f1-score   support

       False       0.84      0.69      0.76      3980
        True       0.62      0.79      0.70      2557

    accuracy                           0.73      6537
   macro avg       0.73      0.74      0.73      6537
weighted avg       0.75      0.73      0.73      6537



In [3]:
match_data

,date,team,opponent,team_score,opponent_score,tournament,city,country,neutral,year,team_rank,team_points,opponent_rank,opponent_points,rank_diff,avg_rank,score_diff,team_won,result,comp
589,1998-01-10,Zambia,Malawi,1,0,COSAFA Cup qualification,Blantyre,Malawi,False,1998,54,1616.70,119,1333.92,-65,86.5,1,True,1,False
10,1998-01-10,Malawi,Zambia,0,1,COSAFA Cup qualification,Blantyre,Malawi,False,1998,119,1333.92,54,1616.70,65,86.5,-1,False,-1,False
4,1998-01-18,Lesotho,Zimbabwe,0,2,COSAFA Cup qualification,Maseru,Lesotho,False,1998,152,1159.80,86,1471.12,66,119.0,-2,False,-1,False
681,1998-01-18,Zimbabwe,Lesotho,2,0,COSAFA Cup qualification,Maseru,Lesotho,False,1998,86,1471.12,152,1159.80,-66,119.0,2,True,1,False
393,1998-01-24,Namibia,South Africa,3,2,COSAFA Cup qualification,Windhoek,Namibia,False,1998,112,1364.70,46,1657.97,66,79.0,1,True,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33723,2018-12-06,Philippines,Vietnam,1,2,AFF Championship,Hanoi,Vietnam,False,2018,149,1172.39,128,1277.79,21,138.5,-1,False,-1,False
33718,2018-12-11,Malaysia,Vietnam,2,2,AFF Championship,Kuala Lumpur,Malaysia,False,2018,160,1057.30,128,1277.79,32,144.0,0,False,0,False
34477,2018-12-11,Vietnam,Malaysia,2,2,AFF Championship,Kuala Lumpur,Malaysia,False,2018,128,1277.79,160,1057.30,-32,144.0,0,False,0,False
33720,2018-12-15,Malaysia,Vietnam,0,1,AFF Championship,Hanoi,Vietnam,False,2018,160,1057.30,128,1277.79,32,144.0,-1,False,-1,False


In [4]:
#for y in wc_start:
#    print(y)

#y = 2022
#df = wc_rankings("1998-01-01", wc_start[y])
#df.to_csv(f"data/rankings{str(y)[-2:]}.csv", index=False)

In [5]:
s = Simulation(YEAR, logreg, match_data)

In [6]:
s.simulate_tournament(True, False)

2-2: Uruguay draws Russia: [0.59, 0.26, 0.15] ->  1
3-0: Uruguay beats Saudi Arabia: [0.74, 0.17, 0.09] ->  3
3-1: Uruguay beats Egypt: [0.64, 0.23, 0.13] ->  3
3-1: Russia beats Saudi Arabia: [0.51, 0.28, 0.21] ->  3
1-3: Russia loses to Egypt: [0.4, 0.31, 0.29] ->  0
1-3: Saudi Arabia loses to Egypt: [0.26, 0.29, 0.45] ->  0
[('Uruguay', 7, 5, 8), ('Egypt', 6, 2, 7), ('Russia', 4, 0, 6), ('Saudi Arabia', 0, -7, 2)] draws: 1

2-0: Spain beats Portugal: [0.4, 0.35, 0.25] ->  3
2-0: Spain beats Iran: [0.67, 0.23, 0.1] ->  3
3-0: Spain beats Morocco: [0.78, 0.15, 0.07] ->  3
1-1: Portugal draws Iran: [0.6, 0.26, 0.14] ->  1
2-0: Portugal beats Morocco: [0.71, 0.2, 0.09] ->  3
0-1: Iran loses to Morocco: [0.46, 0.3, 0.24] ->  0
[('Spain', 9, 7, 7), ('Portugal', 4, 0, 3), ('Morocco', 3, -4, 1), ('Iran', 1, -3, 1)] draws: 1

3-0: France beats Denmark: [0.53, 0.29, 0.18] ->  3
0-2: France loses to Peru: [0.45, 0.33, 0.22] ->  0
0-0: France draws Australia: [0.69, 0.21, 0.1] ->  1
2-4: Denmar

('Spain', 'Brazil', 'Germany', 'Peru')

In [7]:
s.simulate_tournament(False, True)

Group Winners: [['Saudi Arabia', 'Uruguay'], ['Spain', 'Portugal'], ['France', 'Peru'], ['Argentina', 'Iceland'], ['Brazil', 'Serbia'], ['Germany', 'Mexico'], ['England', 'Belgium'], ['Senegal', 'Colombia']]

R16:  ['Saudi Arabia', 'Portugal', 'France', 'Iceland', 'Brazil', 'Mexico', 'England', 'Colombia', 'Spain', 'Uruguay', 'Argentina', 'Peru', 'Germany', 'Serbia', 'Senegal', 'Belgium']
qf: ['Portugal', 'France', 'Mexico', 'England', 'Spain', 'Peru', 'Serbia', 'Belgium']
sf: ['Portugal', 'England', 'Spain', 'Belgium']
B  : ['Portugal', 'Spain']
F  : ['England', 'Belgium']
third: ['Spain']
champ: ['Belgium']


('Belgium', 'England', 'Spain', 'Portugal')

In [8]:
s.simulate_knockout_round(s.get_knockout_round(False), pr=False)
print(s.get_result("r16"))
print(s.get_result("qf"))
print(s.get_result("sf"))
print(s.get_result("final"))
print(s.get_result("summary"))

['Uruguay', 'Spain', 'France', 'Croatia', 'Brazil', 'Mexico', 'Panama', 'Colombia', 'Portugal', 'Russia', 'Nigeria', 'Peru', 'Germany', 'Switzerland', 'Senegal', 'England']
['Spain', 'Croatia', 'Brazil', 'Colombia', 'Portugal', 'Peru', 'Germany', 'England']
['Spain', 'Brazil', 'Peru', 'Germany']
['Brazil', 'Peru']
['Peru', 'Brazil', 'Germany', 'Spain']


In [9]:
s.get_knockout_round(True)

2-3: Uruguay loses to Russia: [0.59, 0.26, 0.15] ->  0
2-1: Uruguay beats Saudi Arabia: [0.74, 0.17, 0.09] ->  3
1-4: Uruguay loses to Egypt: [0.64, 0.23, 0.13] ->  0
1-0: Russia beats Saudi Arabia: [0.51, 0.28, 0.21] ->  3
0-1: Russia loses to Egypt: [0.4, 0.31, 0.29] ->  0
1-1: Saudi Arabia draws Egypt: [0.26, 0.29, 0.45] ->  1
[('Egypt', 7, 4, 6), ('Russia', 6, 1, 4), ('Uruguay', 3, -3, 5), ('Saudi Arabia', 1, -2, 2)] draws: 1

2-0: Spain beats Portugal: [0.4, 0.35, 0.25] ->  3
1-1: Spain draws Iran: [0.67, 0.23, 0.1] ->  1
3-1: Spain beats Morocco: [0.78, 0.15, 0.07] ->  3
2-1: Portugal beats Iran: [0.6, 0.26, 0.14] ->  3
0-1: Portugal loses to Morocco: [0.71, 0.2, 0.09] ->  0
3-2: Iran beats Morocco: [0.46, 0.3, 0.24] ->  3
[('Spain', 7, 4, 6), ('Iran', 4, 0, 5), ('Morocco', 3, -2, 4), ('Portugal', 3, -2, 2)] draws: 1

4-1: France beats Denmark: [0.53, 0.29, 0.18] ->  3
3-3: France draws Peru: [0.45, 0.33, 0.22] ->  1
1-0: France beats Australia: [0.69, 0.21, 0.1] ->  3
1-2: Denma

[['Egypt', 'Russia'],
 ['Spain', 'Iran'],
 ['France', 'Peru'],
 ['Argentina', 'Nigeria'],
 ['Switzerland', 'Brazil'],
 ['Germany', 'South Korea'],
 ['Belgium', 'England'],
 ['Colombia', 'Japan']]

In [10]:
for year in [2022]: #[2010, 2014, 2018, 2022]:
    common_winners(year, 10000)

Most common winners of 2022 World Cup...
Brazil: 28.2%
France: 12.7%
Argentina: 12.5%
Belgium: 8.7%
Portugal: 8.0%
Spain: 7.4%
Germany: 4.7%
Switzerland: 3.7%
Uruguay: 3.3%
Denmark: 2.7%


In [11]:
def evaluate_groups(group, iterations):
    winners_dict = {}
    finalists_dict = {}
    for i in range(iterations):
        gw = s.get_knockout_round_group(group, False) #gw = get_knockout_round(groups_list[YEAR], False)
        if gw[0] in winners_dict.keys():
            winners_dict[gw[0]] += 1
        else:
            winners_dict[gw[0]] = 1
            
        gw_set = frozenset(gw)
        if gw_set in finalists_dict.keys():
            finalists_dict[gw_set] += 1
        else:
            finalists_dict[gw_set] = 1


    print(f"Results for {YEAR} World Cup...")

    print("Most common group winners")        
    for team, num in sorted(winners_dict.items(), key=lambda x:x[1], reverse=True):
        print(f"{team}: {round((num / iterations) * 100, 1)}%")

    print("\nMost common final fours")
    for teams, num in sorted(finalists_dict.items(), key=lambda x:x[1], reverse=True):
        print(f"{list(teams)}: {round((num / iterations) * 100, 1)}%")

#for group in get_groups(year).values():
#    print(group)
#    evaluate_groups(group, 100)
#    print('\n\n')

In [12]:
rankings[rankings["Year"] == YEAR][:15]
# NOTE: 199/900 world cup games end in draw  # 0.2211111111111111
# NOTE: 10084/43752 of all games end in draw. # 0.2304808923020662

,Rank,Team,Score,Year
3961,1,Brazil,2121.91,2018
3962,2,Germany,2099.42,2018
3963,3,Spain,2039.00,2018
3964,4,France,1986.47,2018
3965,5,Argentina,1984.39,2018
3966,6,Portugal,1983.57,2018
3967,7,England,1937.57,2018
3968,8,Colombia,1929.62,2018
3969,9,Belgium,1928.18,2018
3970,10,Italy,1901.35,2018
